In [17]:
import numpy as np
import time
from tqdm import tqdm

In [13]:
n=5000
p=500
X=np.random.rand(n,p)
y=np.random.rand(n)

# Original method

In [18]:
def k_o(X,Y,index_o):
    inner_o=np.dot(X*index_o[None,:],Y.T)
    rest=1-index_o
    inner_rest=np.dot(X*rest[None,:],Y.T)
    return (1+inner_o)**2-1+2*inner_rest*inner_o




start=time.time()
activations=[]
for index_o in tqdm(np.eye(p)):
    activation=np.einsum('ij,i,j->',k_o(X,X,index_o),y,y)
    activations.append(activation)
print(time.time()-start)

100%|██████████| 500/500 [02:32<00:00,  3.27it/s]

152.89659094810486


# Speedup

In [20]:
def activation(K,y,xo):
    vo=y*xo
    wo=y*xo**2
    return np.einsum('ij,i,j->',K,vo,vo)-np.sum(wo)**2

start=time.time()
K=2*(1+np.dot(X,X.T))
activations_2=[]
for index_o in tqdm(np.eye(p)):
    xo=np.sum(X*index_o[None,:],axis=1)
    activation_=activation(K,y,xo)
    activations_2.append(activation_)
print(time.time()-start)

100%|██████████| 500/500 [00:17<00:00, 29.35it/s]

17.14207887649536


In [21]:
np.allclose(activations,activations_2)

True

Note that this speedup becomes even larger when implemented in JAX (thanks to JIT compilation and GPU parralelization), because the first method cannot be computed on GPU due to the intense memory requirement. 